In [ ]:
# Version 2: make the mpl dataset for right hand

# calc every hand pose joint value, mapping joint keypoints and record the image & data
%matplotlib inline

import sys
sys.path.append("/home/jade/DRL/codes/MuJoCo/Hand_Control/drive_hand")
sys.path.append("/home/jade/DRL/codes/MuJoCo/Hand_Control/data_importer/data_importer")
from mpl_importer import MPL
import numpy as np
from hand_mapping import hand_keypoints_mapping
from joint_position_calculator import get_pose_new4 as get_pose_new

from numpy.testing import assert_array_equal,assert_almost_equal
from mujoco_py import MjSim, MjViewer, load_model_from_xml, load_model_from_path, MjSimState, ignore_mujoco_warnings
import mujoco_py
import cv2
from skimage.io import imsave, imshow
import matplotlib.pyplot as plt
import json
from funtest.test_pathlib import first_try
import time


# prepare the mujcoco simulation environment
xml_path = "/home/jade/DRL/codes/MuJoCo/Resources/xml_model/MPL/robot_hand.xml"
model = load_model_from_path(xml_path)
sim = MjSim(model)
viewer = MjViewer(sim)

def print_hand_joint_pose(sim):
    print("hand pose:")
    print("wrist=", sim.data.get_joint_qpos("wrist_PRO"), sim.data.get_joint_qpos("wrist_UDEV"), sim.data.get_joint_qpos("wrist_FLEX"))
    print("thumb=", sim.data.get_joint_qpos("thumb_ABD"), sim.data.get_joint_qpos("thumb_MCP"), sim.data.get_joint_qpos("thumb_PIP"), sim.data.get_joint_qpos("thumb_DIP"))
    print("index=", sim.data.get_joint_qpos("index_ABD"), sim.data.get_joint_qpos("index_MCP"), sim.data.get_joint_qpos("index_PIP"), sim.data.get_joint_qpos("index_DIP"))
    print("middle=", sim.data.get_joint_qpos("middle_ABD"), sim.data.get_joint_qpos("middle_MCP"), sim.data.get_joint_qpos("middle_PIP"), sim.data.get_joint_qpos("middle_DIP"))
    print("ring=", sim.data.get_joint_qpos("ring_ABD"), sim.data.get_joint_qpos("ring_MCP"), sim.data.get_joint_qpos("ring_PIP"), sim.data.get_joint_qpos("ring_DIP"))
    print("pinky=", sim.data.get_joint_qpos("pinky_ABD"), sim.data.get_joint_qpos("pinky_MCP"), sim.data.get_joint_qpos("pinky_PIP"), sim.data.get_joint_qpos("pinky_DIP"))

print_hand_joint_pose(sim)

# get the ID of mechanical hand MPL in mujoco env
wrist_j0 = sim.model.get_joint_qpos_addr("wrist_PRO")
wrist_j1 = sim.model.get_joint_qpos_addr("wrist_UDEV")
wrist_j2 = sim.model.get_joint_qpos_addr("wrist_FLEX")

thumb_j0 = sim.model.get_joint_qpos_addr("thumb_ABD")
thumb_j1 = sim.model.get_joint_qpos_addr("thumb_MCP")
thumb_j2 = sim.model.get_joint_qpos_addr("thumb_PIP")
thumb_j3 = sim.model.get_joint_qpos_addr("thumb_DIP")

index_j0 = sim.model.get_joint_qpos_addr("index_ABD")
index_j1 = sim.model.get_joint_qpos_addr("index_MCP")
index_j2 = sim.model.get_joint_qpos_addr("index_PIP")
index_j3 = sim.model.get_joint_qpos_addr("index_DIP")

middle_j0 = sim.model.get_joint_qpos_addr("middle_ABD")
middle_j1 = sim.model.get_joint_qpos_addr("middle_MCP")
middle_j2 = sim.model.get_joint_qpos_addr("middle_PIP")
middle_j3 = sim.model.get_joint_qpos_addr("middle_DIP")

ring_j0 = sim.model.get_joint_qpos_addr("ring_ABD")
ring_j1 = sim.model.get_joint_qpos_addr("ring_MCP")
ring_j2 = sim.model.get_joint_qpos_addr("ring_PIP")
ring_j3 = sim.model.get_joint_qpos_addr("ring_DIP")

pinky_j0 = sim.model.get_joint_qpos_addr("pinky_ABD")
pinky_j1 = sim.model.get_joint_qpos_addr("pinky_MCP")
pinky_j2 = sim.model.get_joint_qpos_addr("pinky_PIP")
pinky_j3 = sim.model.get_joint_qpos_addr("pinky_DIP")


# loop for different viewport to make nine datasets
for vp in range(0,9):
    # prepare the image setting for the MPL hand in mujoco env
    frame_size = [128, 128]
    camera_name = "mpl_camera{}".format(vp)
    device_id = 0

    # prepare the save file I/O
    # sys.path.append("/home/jade/DRL/codes/MuJoCo/Hand_Control/")
    file_hand_ground_truth = open('../mpl/mpl_mujoco/depth_mpl{}/hand_groundtruth.txt'.format(vp),'w+')
    file_hand_mapping_keypoints = open('../mpl/mpl_mujoco/depth_mpl{}/hand_mapping_keyponts.txt'.format(vp),'w+')
    file_hand_calculated_joints = open('../mpl/mpl_mujoco/depth_mpl{}/hand_calculated_joints.txt'.format(vp),'w+')

    # make the iteration of the hand joints calculation through groundtruth dataset
    for seq in range(0,9):
        mpl = MPL(seq)
        for idx in range(0,8500):
            try:
                # import groundtruth hand keypoints data
                # mpl = MPL(seq)
                finger_data = mpl.data[idx].gtorig
                # print("hand groundtruth keypoints:\n%s"%finger_data)

                # calc the mechanical hand keypoint mapping data and mechanical hand joints
                finger_data_mapping = hand_keypoints_mapping.calc_model_mpl_keypoints(finger_data)
                # print("mechanical hand mapping keypoints:\n%s"%finger_data_mapping)
                hand_pose = get_pose_new.get_hand_joint_pose(finger_data)
                # print("calculated hand joints:\n%s"%hand_pose)

                # change and reset the state of mechanical hand MPL in mujoco env
                sim_state = sim.get_state()
                sim_state.qpos[wrist_j0] = 0.0
                sim_state.qpos[wrist_j1] = 0.0
                sim_state.qpos[wrist_j2] = 0.0

                sim_state.qpos[thumb_j0] = hand_pose['thumb'][0]
                sim_state.qpos[thumb_j1] = hand_pose['thumb'][1]
                sim_state.qpos[thumb_j2] = hand_pose['thumb'][2]
                sim_state.qpos[thumb_j3] = hand_pose['thumb'][3]

                sim_state.qpos[index_j0] = hand_pose['index'][0]
                sim_state.qpos[index_j1] = hand_pose['index'][1]
                sim_state.qpos[index_j2] = hand_pose['index'][2]
                sim_state.qpos[index_j3] = hand_pose['index'][3]

                sim_state.qpos[middle_j0] = hand_pose['middle'][0]
                sim_state.qpos[middle_j1] = hand_pose['middle'][1]
                sim_state.qpos[middle_j2] = hand_pose['middle'][2]
                sim_state.qpos[middle_j3] = hand_pose['middle'][3]

                sim_state.qpos[ring_j0] = hand_pose['ring'][0]
                sim_state.qpos[ring_j1] = hand_pose['ring'][1]
                sim_state.qpos[ring_j2] = hand_pose['ring'][2]
                sim_state.qpos[ring_j3] = hand_pose['ring'][3]

                sim_state.qpos[pinky_j0] = hand_pose['pinky'][0]
                sim_state.qpos[pinky_j1] = hand_pose['pinky'][1]
                sim_state.qpos[pinky_j2] = hand_pose['pinky'][2]
                sim_state.qpos[pinky_j3] = hand_pose['pinky'][3]

                sim.set_state(sim_state)

                # begin the simulation step and render
                sim.forward()
                sim.step()
                # print("move hand to", hand_pose)
                # print_hand_joint_pose(sim)

                # check whether the components of MPL happened collision
                assert sim.data.ncon == 0, 'MPL hand Collision Detected after stepping!'

                # render the mujoco scene and save the color/depth image of MPL into the file
                rgb_image, depth_image = sim.render(width = frame_size[0], height = frame_size[1], depth = True, mode = 'offscreen', camera_name = camera_name, device_id = device_id)

                # check whether the components of MPL happened collision
                assert sim.data.ncon == 0, 'MPL hand Collision Detected after rendering!'

                # format the finger data and write them into file
                str_finger_data = ','.join(map(str, finger_data))
                file_hand_ground_truth.write("P{}_".format(seq) + str(idx) + ":" + str_finger_data + "\n")
                str_finger_data_mapping = ','.join(map(str, finger_data_mapping))
                file_hand_mapping_keypoints.write("P{}_".format(seq) + str(idx) + ":" + str_finger_data_mapping + "\n")
                str_hand_pose = str(hand_pose)
                file_hand_calculated_joints.write("P{}_".format(seq) + str(idx) + ":" + str_hand_pose + "\n")
                                                  
                # we set the camera euler angle as [0.785398 0 0]//x axis rotate 45 degree
                # rotate the image along the u axis(horizontally flip) and save the rgb image
                rgb_image = cv2.flip(rgb_image, 0)
                # print(np.shape(rgb_image))
                cv2.imwrite("../mpl/mpl_mujoco/depth_mpl{}/rgb_P{}_{}.jpg".format(vp, seq, idx), rgb_image)
                # cv2.imshow(rgb_image)
                # plt.imshow(rgb_image)
                # plt.show()

                # rotate the image along the u axis(horizontally flip) and save the depth image
                depth_image = cv2.flip(depth_image, 0)
                # print(np.shape(depth_image))
                # translate the value range into range[0,255]
                dmin = np.amin(depth_image)
                dmax = np.amax(depth_image)
                scale = (dmax-dmin) / 255.0
                depth_image = (depth_image - dmin) / scale
                cv2.imwrite("../mpl/mpl_mujoco/depth_mpl{}/depth_P{}_{}.png".format(vp, seq, idx), depth_image)
                # cv2.imshow(depth_image)
                # plt.imshow(depth_image)
                # plt.show()

                # time.sleep(0.002)

            except:
                # raise
                # print("Components of MPL Hand in MuJoCo scene have collided, so we exclude the data sample and keep going")
                # pass
                continue

            else:
                print("No collision happened in MuJoCo scene, so let's run the next data sample")

# finally 
file_hand_ground_truth.close()
file_hand_mapping_keypoints.close()
file_hand_calculated_joints.close()
print("haha, all dataset sample have been checked, well done!")

    

In [ ]:
# Version 2: crop the human_msra15 dataset into 100*100 size(normalized to 255)

import os
import os.path
from PIL import Image
import numpy as np


# loop for different viewpoint to make nine datasets
for vp in range(0,9):
    file_path = "../mpl/mpl_mujoco/depth_mpl{}/".format(vp)
    file_name = os.listdir(file_path)
    
    for fn in file_name:
        if fn[-3:] == 'png':
            image = Image.open(file_path + str(fn))
            width,hight = image.size
            # print("width:{}, hight:{}".format(width, hight))
            # print(np.array(image).shape)

            # make sure the crop size
            img = np.array(image)
            top = 0
            bottom = 0
            left = 0
            right = 0

            for m in range(0,128,1):
                if (img[m,:].max() - img[m,:].min())>200:
                    top = m
                    # print(top)
                    break

            for n in range(127,-1,-1):
                if (img[n,:].max() - img[n,:].min())>200:
                    bottom = n
                    # print(bottom)
                    break

            for i in range(0,128,1):
                if (img[:,i].max() - img[:,i].min())>200:
                    left = i
                    # print(left)
                    break

            for j in range(127,-1,-1):
                if (img[:,j].max() - img[:,j].min())>200:
                    right = j
                    # print(right)
                    break

            center_y = (top + bottom)//2
            center_x = (left + right)//2

            top_crop_temp = center_y - 50
            bottom_crop_temp = center_y + 50
            left_crop_temp = center_x - 50
            right_crop_temp = center_x + 50

            if top_crop_temp <=0:
                top_crop = 0
                bottom_crop = 100
            else:
                top_crop = top_crop_temp
                bottom_crop = bottom_crop_temp

            if left_crop_temp <=0:
                left_crop = 0
                right_crop = 100
            else:
                left_crop = left_crop_temp
                right_crop = right_crop_temp

            if bottom_crop_temp >=128:
                bottom_crop = 127
                top_crop = 27
            else:
                top_crop = top_crop_temp
                bottom_crop = bottom_crop_temp

            if right_crop_temp >=128:
                right_crop = 127
                left_crop = 27
            else:
                left_crop = left_crop_temp
                right_crop = right_crop_temp

            # print("top:{},bottom:{},left:{},right:{}".format(top_crop,bottom_crop,left_crop,right_crop))
            img_crop = image.crop((left_crop,top_crop,right_crop,bottom_crop))
            width_crop,hight_crop = img_crop.size
            # print("width_crop:{}, hight_crop:{}".format(width_crop, hight_crop))

            img_resize = img_crop.resize((100,100))
            img_resize.save('../mpl/mpl_crop/depth_mpl{}/{}_{}.png'.format(vp, fn[0:-4], 'crop'))



# Appendix

In [ ]:
# Version 1: crop the human_msra15 dataset into 100*100 size

import os
import os.path
from PIL import Image
import numpy as np


file_path = "../mpl/mpl_mujoco/depth_mpl0/"
file_name = os.listdir(file_path)

for fn in file_name:
    if fn[-3:] == 'png':
        image = Image.open(file_path + str(fn))
        width,hight = image.size
        print("width:{}, hight:{}".format(width, hight))
        print(np.array(image).shape)
        
        # make sure the crop size
        img = np.array(image)
        top = 0
        bottom = 0
        left = 0
        right = 0

        for m in range(0,128,1):
            if (img[m,:].max() - img[m,:].min())>200:
                top = m
                print(top)
                break

        for n in range(127,-1,-1):
            if (img[n,:].max() - img[n,:].min())>200:
                bottom = n
                print(bottom)
                break

        for i in range(0,128,1):
            if (img[:,i].max() - img[:,i].min())>200:
                left = i
                print(left)
                break

        for j in range(127,-1,-1):
            if (img[:,j].max() - img[:,j].min())>200:
                right = j
                print(right)
                break
        
        center_y = (top + bottom)//2
        center_x = (left + right)//2
        
        top_crop_temp = center_y - 50
        bottom_crop_temp = center_y + 50
        left_crop_temp = center_x - 50
        right_crop_temp = center_x + 50
        
        if top_crop_temp <=0:
            top_crop = 0
            bottom_crop = 100
        else:
            top_crop = top_crop_temp
            bottom_crop = bottom_crop_temp

        if left_crop_temp <=0:
            left_crop = 0
            right_crop = 100
        else:
            left_crop = left_crop_temp
            right_crop = right_crop_temp

        if bottom_crop_temp >=128:
            bottom_crop = 127
            top_crop = 27
        else:
            top_crop = top_crop_temp
            bottom_crop = bottom_crop_temp

        if right_crop_temp >=128:
            right_crop = 127
            left_crop = 27
        else:
            left_crop = left_crop_temp
            right_crop = right_crop_temp
            
        print("top:{},bottom:{},left:{},right:{}".format(top_crop,bottom_crop,left_crop,right_crop))
        img_crop = image.crop((left_crop,top_crop,right_crop,bottom_crop))
        width_crop,hight_crop = img_crop.size
        print("width_crop:{}, hight_crop:{}".format(width_crop, hight_crop))
        
        img_resize = img_crop.resize((100,100))
        img_resize.save('../mpl/mpl_crop/depth_mpl0/{}_{}.png'.format(fn[0:-4],'crop'))
        
        

In [ ]:
# Version 1: make the mpl dataset for right hand

# calc every hand pose joint value, mapping joint keypoints and record the image & data
%matplotlib inline

import sys
sys.path.append("/home/jade/DRL/codes/MuJoCo/Hand_Control/drive_hand")
sys.path.append("/home/jade/DRL/codes/MuJoCo/Hand_Control/data_importer/data_importer")
import numpy as np
from data_importer.importer import Anyone
from hand_mapping import hand_keypoints_mapping
from joint_position_calculator import get_pose_new4

from numpy.testing import assert_array_equal,assert_almost_equal
from mujoco_py import MjSim, MjViewer, load_model_from_xml, load_model_from_path, MjSimState, ignore_mujoco_warnings
import mujoco_py
import cv2
from skimage.io import imsave, imshow
import matplotlib.pyplot as plt
import json
from funtest.test_pathlib import first_try
import time


# prepare the save file I/O
# sys.path.append("/home/jade/DRL/codes/MuJoCo/Hand_Control/")
file_hand_ground_truth = open('../mpl/mpl_mujoco/depth_mpl8/hand_groundtruth.txt','w+')
file_hand_mapping_keypoints = open('../mpl/mpl_mujoco/depth_mpl8/hand_mapping_keyponts.txt','w+')
file_hand_calculated_joints = open('../mpl/mpl_mujoco/depth_mpl8/hand_calculated_joints.txt','w+')

# prepare the mujcoco simulation environment
xml_path = "/home/jade/DRL/codes/MuJoCo/Resources/xml_model/MPL/robot_hand.xml"
model = load_model_from_path(xml_path)
sim = MjSim(model)
viewer = MjViewer(sim)

def print_hand_joint_pose(sim):
    print("hand pose:")
    print("wrist=", sim.data.get_joint_qpos("wrist_PRO"), sim.data.get_joint_qpos("wrist_UDEV"), sim.data.get_joint_qpos("wrist_FLEX"))
    print("thumb=", sim.data.get_joint_qpos("thumb_ABD"), sim.data.get_joint_qpos("thumb_MCP"), sim.data.get_joint_qpos("thumb_PIP"), sim.data.get_joint_qpos("thumb_DIP"))
    print("index=", sim.data.get_joint_qpos("index_ABD"), sim.data.get_joint_qpos("index_MCP"), sim.data.get_joint_qpos("index_PIP"), sim.data.get_joint_qpos("index_DIP"))
    print("middle=", sim.data.get_joint_qpos("middle_ABD"), sim.data.get_joint_qpos("middle_MCP"), sim.data.get_joint_qpos("middle_PIP"), sim.data.get_joint_qpos("middle_DIP"))
    print("ring=", sim.data.get_joint_qpos("ring_ABD"), sim.data.get_joint_qpos("ring_MCP"), sim.data.get_joint_qpos("ring_PIP"), sim.data.get_joint_qpos("ring_DIP"))
    print("pinky=", sim.data.get_joint_qpos("pinky_ABD"), sim.data.get_joint_qpos("pinky_MCP"), sim.data.get_joint_qpos("pinky_PIP"), sim.data.get_joint_qpos("pinky_DIP"))

print_hand_joint_pose(sim)

# get the ID of mechanical hand MPL in mujoco env
wrist_j0 = sim.model.get_joint_qpos_addr("wrist_PRO")
wrist_j1 = sim.model.get_joint_qpos_addr("wrist_UDEV")
wrist_j2 = sim.model.get_joint_qpos_addr("wrist_FLEX")

thumb_j0 = sim.model.get_joint_qpos_addr("thumb_ABD")
thumb_j1 = sim.model.get_joint_qpos_addr("thumb_MCP")
thumb_j2 = sim.model.get_joint_qpos_addr("thumb_PIP")
thumb_j3 = sim.model.get_joint_qpos_addr("thumb_DIP")

index_j0 = sim.model.get_joint_qpos_addr("index_ABD")
index_j1 = sim.model.get_joint_qpos_addr("index_MCP")
index_j2 = sim.model.get_joint_qpos_addr("index_PIP")
index_j3 = sim.model.get_joint_qpos_addr("index_DIP")

middle_j0 = sim.model.get_joint_qpos_addr("middle_ABD")
middle_j1 = sim.model.get_joint_qpos_addr("middle_MCP")
middle_j2 = sim.model.get_joint_qpos_addr("middle_PIP")
middle_j3 = sim.model.get_joint_qpos_addr("middle_DIP")

ring_j0 = sim.model.get_joint_qpos_addr("ring_ABD")
ring_j1 = sim.model.get_joint_qpos_addr("ring_MCP")
ring_j2 = sim.model.get_joint_qpos_addr("ring_PIP")
ring_j3 = sim.model.get_joint_qpos_addr("ring_DIP")

pinky_j0 = sim.model.get_joint_qpos_addr("pinky_ABD")
pinky_j1 = sim.model.get_joint_qpos_addr("pinky_MCP")
pinky_j2 = sim.model.get_joint_qpos_addr("pinky_PIP")
pinky_j3 = sim.model.get_joint_qpos_addr("pinky_DIP")

# prepare the image setting for the MPL hand in mujoco env
frame_size = [128, 128]
camera_name = "mpl_camera8"
device_id = 0

# make the iteration of the hand joints calculation through groundtruth dataset
for idx in range(1,50):
    try:
        # import groundtruth hand keypoints data
        ay = Anyone(idx)
        finger_data = ay.data.gtorig
        print("hand groundtruth keypoints:\n%s"%finger_data)

        # calc the mechanical hand keypoint mapping data and mechanical hand joints
        finger_data_mapping = hand_keypoints_mapping.calc_model_mpl_keypoints(finger_data)
        print("mechanical hand mapping keypoints:\n%s"%finger_data_mapping)
        hand_pose = get_pose_new4.get_hand_joint_pose(finger_data)
        print("calculated hand joints:\n%s"%hand_pose)

        # change and reset the state of mechanical hand MPL in mujoco env
        sim_state = sim.get_state()
        sim_state.qpos[wrist_j0] = 0.0
        sim_state.qpos[wrist_j1] = 0.0
        sim_state.qpos[wrist_j2] = 0.0

        sim_state.qpos[thumb_j0] = hand_pose['thumb'][0]
        sim_state.qpos[thumb_j1] = hand_pose['thumb'][1]
        sim_state.qpos[thumb_j2] = hand_pose['thumb'][2]
        sim_state.qpos[thumb_j3] = hand_pose['thumb'][3]

        sim_state.qpos[index_j0] = hand_pose['index'][0]
        sim_state.qpos[index_j1] = hand_pose['index'][1]
        sim_state.qpos[index_j2] = hand_pose['index'][2]
        sim_state.qpos[index_j3] = hand_pose['index'][3]

        sim_state.qpos[middle_j0] = hand_pose['middle'][0]
        sim_state.qpos[middle_j1] = hand_pose['middle'][1]
        sim_state.qpos[middle_j2] = hand_pose['middle'][2]
        sim_state.qpos[middle_j3] = hand_pose['middle'][3]

        sim_state.qpos[ring_j0] = hand_pose['ring'][0]
        sim_state.qpos[ring_j1] = hand_pose['ring'][1]
        sim_state.qpos[ring_j2] = hand_pose['ring'][2]
        sim_state.qpos[ring_j3] = hand_pose['ring'][3]

        sim_state.qpos[pinky_j0] = hand_pose['pinky'][0]
        sim_state.qpos[pinky_j1] = hand_pose['pinky'][1]
        sim_state.qpos[pinky_j2] = hand_pose['pinky'][2]
        sim_state.qpos[pinky_j3] = hand_pose['pinky'][3]

        sim.set_state(sim_state)

        # begin the simulation step and render
        sim.forward()
        sim.step()
        print("move hand to", hand_pose)
        print_hand_joint_pose(sim)
        
        # check whether the components of MPL happened collision
        assert sim.data.ncon == 0, 'MPL hand Collision Detected after stepping!'

        # render the mujoco scene and save the color/depth image of MPL into the file
        rgb_image, depth_image = sim.render(width = frame_size[0], height = frame_size[1], depth = True, mode = 'offscreen', camera_name = camera_name, device_id = device_id)

        # check whether the components of MPL happened collision
        assert sim.data.ncon == 0, 'MPL hand Collision Detected after rendering!'
        
        # format the finger data and write them into file
        str_finger_data = ','.join(map(str, finger_data))
        file_hand_ground_truth.write(str(idx) + ":" + str_finger_data + "\n")
        str_finger_data_mapping = ','.join(map(str, finger_data_mapping))
        file_hand_mapping_keypoints.write(str(idx) + ":" + str_finger_data_mapping + "\n")
        # str_hand_pose = map(str, hand_pose)
        str_hand_pose = str(hand_pose)
        file_hand_calculated_joints.write(str(idx) + ":" + str_hand_pose + "\n")
        
        # we set the camera euler angle as [0.785398 0 0]//x axis rotate 45 degree
        # rotate the image along the u axis(horizontally flip) and save the rgb image
        rgb_image = cv2.flip(rgb_image, 0)
        print(np.shape(rgb_image))
        cv2.imwrite("../mpl/mpl_mujoco/depth_mpl8/rgb_P0_{}.jpg".format(idx), rgb_image)
        # cv2.imshow(rgb_image)
        plt.imshow(rgb_image)
        plt.show()

        # rotate the image along the u axis(horizontally flip) and save the depth image
        depth_image = cv2.flip(depth_image, 0)
        print(np.shape(depth_image))
        # translate the value range into range[0,255]
        dmin = np.amin(depth_image)
        dmax = np.amax(depth_image)
        scale = (dmax-dmin) / 255.0
        depth_image = (depth_image - dmin) / scale
        cv2.imwrite("../mpl/mpl_mujoco/depth_mpl8/depth_P0_{}.png".format(idx), depth_image)
        # cv2.imshow(depth_image)
        plt.imshow(depth_image)
        plt.show()

        # time.sleep(0.002)

    except:
        # raise
        print("Components of MPL Hand in MuJoCo scene have collided, so we exclude the data sample and keep going")
        # pass
        continue
    
    else:
        print("No collision happened in MuJoCo scene, so let's run the next data sample")

# finally 
file_hand_ground_truth.close()
file_hand_mapping_keypoints.close()
file_hand_calculated_joints.close()
print("haha, all dataset sample have been checked, well done!")

    